In [2]:
import pandas as pd
import pysam

In [4]:
def filter_cat(sample, path, taxa, contig_min_len, orf_min_percentage):
    tsv_base = f'{path}/cat/{sample}/{sample}_classification'
        with open(f'{tsv_base}.tsv', 'r') as f, open(f'{tsv_base}.refined.tsv', 'w') as out:
        for line in f:
            if len(line.split('\t')) == 12:
                print(line, file=out)
    data = pd.read_csv(f'{tsv_base}.refined.tsv', sep="\t")
    data = data[(data['# contig'].str.extract('NODE_\d+_length_(\d+)_cov_.+').astype('int') >= contig_min_len)[0]]
    data = data[(data.reason.str.extract('based on (\d+)/\d+ ORFs').astype(int)/data.reason.str.extract('based on \d+/(\d+) ORFs').astype(int) >= orf_min_percentage)[0]]
    for taxon in taxa:
        data = data[data.lineage.str.contains(str(taxon))]
    data['sample'] = sample
    
    return data

def process_cat(samples, path, taxa=[4751], contig_min_len=1000, orf_min_percentage=0.2):
    all_data = pd.DataFrame(columns=['# contig', 'classification', 'reason', 'lineage', 'lineage scores',
       'superkingdom', 'phylum', 'class', 'order', 'family', 'genus',
       'species', 'sample'])
    
    for sample in samples:
        data = filter_cat(sample=sample, taxa=taxa, contig_min_len=contig_min_len, orf_min_percentage=orf_min_percentage, path=path)
        all_data = pd.concat([all_data, data])

    genus_counts = all_data[['sample', 'genus']].copy()
    genus_counts['counts'] = genus_counts['genus'].str.split(':').str[1].astype(float)
    genus_counts = genus_counts.groupby(['sample', 'genus']).agg({'counts': 'sum'}).reset_index()
    genus_counts['genus'] = genus_counts['genus'].str.replace(': 1.00', '').str.strip()
    genus_counts['counts'] = genus_counts['counts'].astype(int)
    genus_counts['tool'] = 'cat'
    
    return genus_counts

In [5]:
def filter_its(sample, path):
    base = f'{path}/V350174473_L01_UDB-{sample}_UNITE_public_10.05.2021.sorted.bam'
    bam_file = pysam.AlignmentFile(base, 'rb')
    samples = []
    genus_list = []
    counts_list = []

    for read in bam_file:
        reference_name = read.reference_name
        genus = reference_name.split('g__')[1].split(';')[0]
        samples.append(sample)
        genus_list.append(genus)
        counts_list.append(1)
    bam_file.close()

    data = {'sample': samples, 'genus': genus_list, 'counts': counts_list}
    df = pd.DataFrame(data)
    df = df.groupby(['sample', 'genus']).agg({'counts': 'sum'}).reset_index()
    return df

def process_its(samples, path):
    all_data = pd.DataFrame(columns=["sample", "genus", "counts"])
    for sample in samples:
        df = filter_its(sample, path)
        all_data = pd.concat([all_data, df])
    all_data['tool'] = 'its'
    all_data = all_data.reset_index(drop=True)
    return all_data

In [78]:
def filter_kraken(sample, path, n_top=10000):
    report = f'{path}/V350174473_L01_UDB-{sample}_report.txt'
    df = pd.read_csv(report, delimiter="\t", names=["perc", "counts", "only_reads", "rank", "taxid", "genus"])

    include_rows = False
    filtered_rows = []

    for index, row in df.iterrows():
        if row['taxid'] == 4751:
            include_rows = True
            continue
        if row['rank'] == 'D1':
            include_rows = False
        elif row['rank'] == 'G' and include_rows:
            filtered_rows.append(row)

    filtered_df = pd.DataFrame(filtered_rows)
    filtered_df["sample"] = sample
    filtered_df = filtered_df[["sample", "genus", "counts"]].sort_values(by="counts", ascending=False).reset_index(drop=True)
    filtered_df["tool"] = "kraken2"
    return filtered_df

def process_kraken(samples, path):
    all_data = pd.DataFrame(columns=["sample", "genus", "counts"])
    for sample in samples:
        df = filter_kraken(sample, path)
        all_data = pd.concat([all_data, df])
    all_data = all_data.reset_index(drop=True)
    return all_data